In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab6MlApp") \
    .getOrCreate()

sc = spark.sparkContext

Charger les données cancer_reg (un header est présent, de plus, utiliser l'option inferSchema).

In [0]:
data = spark.read.csv("../../data/cancer_reg.csv", header=True, inferSchema=True)

A l'aide de la fonction dir, afficher les méthodes pouvant être utilisées sur data. Afficher les 10 premières lignes du dataframe ainsi chargé.

In [0]:
dir(data)

In [0]:
data.show(10)

Supprimer du dataframe les colonnes binnedInc, Geography, PctSomeCol18_24, PctEmployed16_Over et PctPrivateCoverageAlone.

In [0]:
data = data.drop(
    "binnedInc",
    "Geography",
    "PctSomeCol18_24",
    "PctEmployed16_Over",
    "PctPrivateCoverageAlone",
)

Repérer la colonne cible (target) et stocker dans des variables la liste des noms des colonnes des variables explicatives d'une part et le nom de la colonne cible d'autre part. 

In [0]:
target_col = "TARGET_deathRate"
feature_cols = data.columns
feature_cols.remove(target_col)

In [0]:
print(feature_cols)

Séparer les données en un jeu d'entrainement et un jeu de test à l'aide de la méthode randomSplit. Choisir une répartition 80%-20%.

In [0]:
train_data, test_data = data.randomSplit([0.7, 0.3], seed=1)

In [0]:
train_data.show(5)

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

Assembler les features dans un vecteur colonne unique à l'aide de VectorAssembler.

In [0]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

Instancier un modèle de régression linéaire en spécifiant les colonnes des variables explicatives et la colonne labellisée.

In [0]:
lr = LinearRegression(featuresCol="features", labelCol=target_col)

Créer une pipeline contenant l'assembleur et du modèle de régression linéaire.

In [0]:
pipeline = Pipeline(stages=[assembler, lr])

Entraîner la pipeline sur le jeu d'entraînement.

In [0]:
res = pipeline.fit(train_data)

Réaliser les prédictions sur le jeu de test. Afficher les 10 premières lignes avec leurs prédictions.

In [0]:
predictions = res.transform(test_data)
predictions.toPandas()

Créer un évaluateur correspondant au problème donné (en identifiant bien la typologie du problème). L'utiliser pour évaluer le rmse du modèle obtenu.

In [0]:
evaluator = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
)
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 20.067126521853375
R2: 0.524183923681123


Reprendre les questions précédentes en rajoutant une étape d'ACP à 10 composantes entre le preprocessing et le modèle.

In [0]:
from pyspark.ml.feature import PCA

In [0]:
# assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
pca = PCA(k=10, inputCol="features", outputCol="pca_features")
lr_2 = LinearRegression(featuresCol="pca_features", labelCol=target_col)
pipeline_2 = Pipeline(stages=[assembler, pca, lr_2])
res_2 = pipeline_2.fit(train_data)
predictions_2 = res_2.transform(test_data)
#evaluator = RegressionEvaluator(
#    labelCol=target_col,
#    predictionCol="prediction",
#)
rmse_2 = evaluator.evaluate(predictions_2, {evaluator.metricName: "rmse"})
r2_2 = evaluator.evaluate(predictions_2, {evaluator.metricName: "r2"})
print(f"RMSE: {rmse_2}")
print(f"R2: {r2_2}")

RMSE: 21.57858699571848
R2: 0.44980740692937615


Conclure quant à l'utilité de l'ACP dans ce problème à dimension raisonnable.

Arrêter la session Spark.

In [ ]:
spark.stop()